In [17]:
import os
import numpy as np
from typing import Optional
from sklearn.model_selection import StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.svm import SVC

model1 = MLPClassifier(
    activation="tanh",
    alpha=2.6e-06,
    beta_1=0.999,
    beta_2=0.9,
    hidden_layer_sizes=(50, 50),
    max_iter=64,
    n_iter_no_change=32,
    validation_fraction=0.0,
    warm_start=True,
)

In [22]:
directory = "./samples"
filename = "base.npy"

samples = 500

path_pos = os.path.join(directory, "base_pos.npy")
path_neg = os.path.join(directory, "base_neg.npy")

if os.path.isfile(path_pos) and os.path.isfile(path_neg):
    base_pos = np.load(path_pos)
    base_neg = np.load(path_neg)

    positives = base_pos[np.random.choice(base_pos.shape[0], int(samples / 2))]
    negatives = base_neg[np.random.choice(base_neg.shape[0], int(samples / 2))]

    base_ready = np.vstack((positives, negatives))

In [23]:
from sklearn.metrics import accuracy_score

cross_validation = StratifiedKFold(n_splits=10, shuffle=True)

y = np.hstack((np.ones(int(samples / 2)), np.zeros(int(samples / 2))))

X = base_ready

y_pred1 = []
y_pred2 = []

for train_index, test_index in cross_validation.split(X, y):
    x_train_fold, x_test_fold = X[train_index], X[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]

    segment_bins_train_fold = x_train_fold[:, 0]
    tq_segment_train_fold = x_train_fold[:, 1]

    segment_bins_test_fold = x_test_fold[:, 0]
    tq_segment_test_fold = x_test_fold[:, 1]

    model1.fit(segment_bins_train_fold, y_train_fold)
    y_pred1 = model1.predict(segment_bins_test_fold)

    model1.fit(tq_segment_train_fold, y_train_fold)
    y_pred2 = model1.predict(tq_segment_test_fold)

    accuracy = accuracy_score(y_test_fold, y_pred1)
    print("y_pred1:", accuracy)

    accuracy = accuracy_score(y_test_fold, y_pred2)
    print("y_pred2:", accuracy)

    combined_labels = np.array(
        [np.argmax(np.bincount([y_pred1[i], y_pred2[i]])) for i in range(len(y_pred1))]
    )
    accuracy = accuracy_score(y_test_fold, combined_labels)
    print("Ensemble Classifier Accuracy:", accuracy)

    print()

y_pred1: 0.96
y_pred2: 0.54
Ensemble Classifier Accuracy: 0.8

y_pred1: 0.96
y_pred2: 0.64
Ensemble Classifier Accuracy: 0.84

y_pred1: 0.94
y_pred2: 0.72
Ensemble Classifier Accuracy: 0.88

y_pred1: 1.0
y_pred2: 0.68
Ensemble Classifier Accuracy: 0.82

y_pred1: 0.94
y_pred2: 0.62
Ensemble Classifier Accuracy: 0.84

y_pred1: 0.98
y_pred2: 0.66
Ensemble Classifier Accuracy: 0.8

y_pred1: 0.96
y_pred2: 0.68
Ensemble Classifier Accuracy: 0.88

y_pred1: 0.98
y_pred2: 0.66
Ensemble Classifier Accuracy: 0.88

y_pred1: 0.98
y_pred2: 0.74
Ensemble Classifier Accuracy: 0.88

y_pred1: 0.98
y_pred2: 0.58
Ensemble Classifier Accuracy: 0.88



In [ ]:
y_test_fold

In [ ]:
# Assuming y_test contains the true labels for the test dataset